In [1]:
#importing libraries
import pandas as pd
import numpy as np
import json
import requests
import datetime
from tqdm import tqdm

In [2]:
# importing subject IDs list having adverse effect
subjectid_ae = requests.get('https://pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/list')
subjectid_ae_json = subjectid_ae.json()
#dataframe for AE
subjectid_ae = pd.DataFrame(subjectid_ae_json['data'], columns = ['subject_id'])

In [3]:
# importing subject IDs list with concomitant medication
subjectid_cm = requests.get('https://pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/list')
subjectid_cm_json= subjectid_cm.json()
subjectid_cm = pd.DataFrame(subjectid_cm_json['data'], columns = ['subject_id'])

In [4]:
#importing remaining data for AE and CM

remainder_ae= requests.get('https://pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/105792/list')
remainder_ae_json= remainder_ae.json()
remainder_ae= pd.DataFrame(remainder_ae_json['data'])

In [5]:
import os
os.chdir('I:\My_projects\saama')
os.getcwd()
remainder_cm= pd.DataFrame(pd.read_excel('cm_data_sample.xlsx'))

In [6]:
# processing data for further cleaning
#removal of unwanted columns and rows

ae = remainder_ae[['siteid', 'subjectid', 'formid', 'aeterm', 'aespid', 'aestdat', 'aeendat', 'aecmgiv']]
ae = ae.dropna(subset=['aestdat', 'aeendat'])

In [7]:
# converting columns into readable format
ae['aestdat'] = pd.to_datetime(ae['aestdat'], errors = 'coerce', format = '%d-%b-%y')
ae['aeendat'] = pd.to_datetime(ae['aeendat'], errors = 'coerce', format = '%d-%b-%y')

In [37]:
# processing data for further cleaning
#removal of unwanted columns and rows

cm = remainder_cm[['SITEID','SUBJID','FORMID','CMTRT','CMAENO','CMSTDAT','CMENDAT','CMAER']]
cm = cm.dropna(subset=['CMSTDAT', 'CMENDAT','CMAENO' ])

In [42]:
# converting columns into readable format
cm['CMSTDAT'] = pd.to_datetime(cm['CMSTDAT'], errors = 'coerce', format = '%d-%b-%y')
cm['CMENDAT'] = pd.to_datetime(cm['CMENDAT'], errors = 'coerce', format = '%d-%b-%y')

In [40]:
final = pd.merge(ae, cm, how = 'outer', left_on = ['subjectid', 'aespid'], right_on = ['SUBJID', 'CMAENO'])
final

,siteid,subjectid,formid,aeterm,aespid,aestdat,aeendat,aecmgiv,SITEID,SUBJID,FORMID,CMTRT,CMAENO,CMSTDAT,CMENDAT,CMAER
0,5538.0,105792.0,38768.0,AST INCREASED,43.0,2018-06-25,2018-07-02,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,5538.0,105792.0,38768.0,ALT INCREASED,44.0,2018-06-22,2018-06-25,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
2,5538.0,105792.0,38768.0,ALT INCREASED,45.0,2018-06-25,2018-06-29,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
3,5538.0,105792.0,38768.0,ALP INCREASED,46.0,2018-05-20,2018-06-25,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
4,5538.0,105792.0,38768.0,ALP INCREASED,47.0,2018-06-25,2018-07-06,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,5178.0,51785008.0,38623.0,DIGOXIN,1.0,NaT,NaT,YES
71,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,5178.0,51785008.0,38623.0,DOCUSATE SODIUM (COLACE),2.0,NaT,NaT,YES
72,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,5134.0,51345002.0,38623.0,Magnesium Sulfate,41.0,NaT,NaT,YES
73,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,5134.0,51345002.0,38623.0,Potassium Chloride 20 mEq in Water 500mL,42.0,NaT,NaT,YES


In [43]:
# discrepancy result for overlapping of Concomitant medications
type4 = final[final.duplicated(['CMTRT', 'CMSTDAT'])]
type4

,siteid,subjectid,formid,aeterm,aespid,aestdat,aeendat,aecmgiv,SITEID,SUBJID,FORMID,CMTRT,CMAENO,CMSTDAT,CMENDAT,CMAER,discrepancy
1,5538.0,105792.0,38768.0,ALT INCREASED,44.0,2018-06-22,2018-06-25,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False
2,5538.0,105792.0,38768.0,ALT INCREASED,45.0,2018-06-25,2018-06-29,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False
3,5538.0,105792.0,38768.0,ALP INCREASED,46.0,2018-05-20,2018-06-25,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False
4,5538.0,105792.0,38768.0,ALP INCREASED,47.0,2018-06-25,2018-07-06,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False
5,5538.0,105792.0,38768.0,MUCOSITIS ORAL,48.0,2018-06-25,2018-06-28,YES,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,5538.0,105792.0,38768.0,MUCOSITIS ORAL,38.0,2018-06-23,2018-06-25,YES,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False
62,5538.0,105792.0,38768.0,THROMBOCYTOPENIA,39.0,2018-06-15,2018-06-18,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False
63,5538.0,105792.0,38768.0,THROMBOCYTOPENIA,40.0,2018-06-18,2018-06-29,NO,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False
64,5538.0,105792.0,38768.0,ENTERITIS INFECTIOUS,41.0,2018-06-19,2018-06-26,YES,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,False


In [44]:
{
  "status": "success",
  "data": "success",
  "request_id": "4696473a-aa70-40c6-a402-c33679cd4afa",
  "message": "query submitted"
}

{'status': 'success',
 'data': 'success',
 'request_id': '4696473a-aa70-40c6-a402-c33679cd4afa',
 'message': 'query submitted'}